# **Research Assistant Python Test**

## Import packages needed for the test

In [1]:
## Import packages
import numpy as np
import pandas as pd
import os
import regex as re
from collections import Counter
from tqdm import tqdm
from rapidfuzz import process, fuzz

# ML packages
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


## Part 1

### Import Data

In [2]:
## Import data
# Set the directory path
directory_path = r"C:\Users\alezi\OneDrive\Escritorio\Test WB\Python"
# directory_path = r"WB_Path"

# Change the current working directory
os.chdir(directory_path)

# Load the data
names_2019_2020 = pd.read_csv("data/ForeignNames_2019_2020.csv")
country_iso = pd.read_csv("data/Country_Name_ISO3.csv")

In [ ]:
# Merge the foreign names dataset with the country ISO3 codes
name_country_data = pd.merge(names_2019_2020 , country_iso, 
                     left_on='foreigncountry_cleaned', 
                     right_on='country_name', 
                     how='left', indicator=True)

# Check the merge results
merge_result_counts = name_country_data['_merge'].value_counts()
print("Merge Results (Value Counts):")
print(merge_result_counts)
print("\n") 
# There are observations that did not merge (left_only)

# Check the countries that did not merge(South Korea, North Korea, Iran, Congo, the Democratic Republic of the, Antigua And Barbuda)
print("Countries that did not merge (left_only):")
unmerged_countries = name_country_data[name_country_data['_merge'] == 'left_only']['foreigncountry_cleaned'].value_counts()
print(unmerged_countries)


In [ ]:
## Fixing the Merge 

# Fix Congo in the names_2019_2020 dataset
names_2019_2020['foreigncountry_cleaned'] = names_2019_2020['foreigncountry_cleaned'].replace('Congo, the Democratic Republic of the','Congo')

# Manual mapping of problematic country names
manual_country_mapping = {
    'Korea, Republic of': 'South Korea',
    'Iran, Islamic Republic of': 'Iran', 
    'Congo, The Democratic Republic of the': 'Congo',
    'Antigua and Barbuda': 'Antigua And Barbuda',
    'Tanzania, United Republic of': 'Tanzania',
}

# Apply the manual mapping to the 'foreigncountry_cleaned' column
country_iso['name_clean'] = country_iso['country_name'].replace(manual_country_mapping)

# Now we can merge based on the mapped country names
name_country_data = pd.merge(names_2019_2020, country_iso, 
                             left_on='foreigncountry_cleaned', 
                             right_on='name_clean', 
                             how='left', indicator=True)

# Check the results
merge_result_counts = name_country_data['_merge'].value_counts()
print("Merge Results (Value Counts):")
print(merge_result_counts)

# Print that if there are no left_only values (left_only = 0), the merge was successful
if merge_result_counts['left_only'] == 0:
    print("\n")
    print("Merge was successful")

### Data Cleaning and Matching

In [107]:
## STOPWORDS PER COUNTRY ##

'''
Define stopwords for cleaning, customized per country.

To enhance the accuracy of cleaning and matching company names, I process the dataset country by country.
For each country, I select the top 10 most frequent words found in the company names specific to that country.
These words are added to a dictionary called 'countries_stopwords', where each key is a country ISO3 code, 
and the value is the list of stopwords for that country.

As I analyze more data, I can expand and refine the stopword lists for each country to further enhance accuracy.
'''


# Initialize an empty dictionary to hold stopwords per country
countries_stopwords = {}

# Group the data by 'country_iso3'
grouped = name_country_data.groupby('country_iso3')

# Iterate over each country group
for country_code, group in grouped:
    # Get the list of company names for this country, ensuring no NaN values
    company_names = group['foreign'].dropna().tolist()
    
    # Initialize a list to hold all words
    all_words = []
    
    # For each company name, extract words
    for name in company_names:
        # Convert to lowercase and remove non-alphabetic characters
        name = name.lower()
        name = re.sub(r'[^a-z\s]', '', name)
        # Split into words
        words = name.split()
        # Add words to the list
        all_words.extend(words)
    
    # Count the frequency of each word
    word_counts = Counter(all_words)
    # Get the top 8 most common words
    top_words = [word for word, count in word_counts.most_common(8)]
    # Add these words to the countries_stopwords dictionary
    countries_stopwords[country_code] = top_words
    
# Add manuallly defined stopwords 
countries_stopwords['USA'].append('technologies')
countries_stopwords['CHN'].append('machinery')


In [ ]:
# Define the function to clean the company names
def clean_company_name(company_name, stopwords):
    if not isinstance(company_name, str):
        return ''
    # Convert to lowercase
    company_name = company_name.lower()
    # Remove non-alphabetic characters
    company_name = re.sub(r'[^a-z\s]', '', company_name)
    # Split into words
    words = company_name.split()
    # Remove stopwords
    cleaned_words = [word for word in words if word not in stopwords]
    # Reconstruct the cleaned name
    cleaned_name = ' '.join(cleaned_words)
    return cleaned_name

# Drop the rows where the company name is missing
name_country_data = name_country_data.dropna(subset=['foreign'])
if name_country_data['foreign'].isnull().sum() == 0:
    print("There are no missing values in the 'foreign' column")

# Apply the cleaning function to the DataFrame using country-specific stopwords
name_country_data['cleaned_name'] = name_country_data.apply(
    lambda row: clean_company_name(
        row['foreign'],
        countries_stopwords.get(row['country_iso3'], [])
    ),
    axis=1
)

In [89]:
## RapidFuzz Functions ##

# Function to apply fuzzy matching within the same country using RapidFuzz
def fuzzy_match_within_country(df, similarity_threshold=85, min_length=4):
    # Use unique cleaned names to avoid duplicates
    unique_cleaned_names = list(set(df['cleaned_name'].tolist()))
    name_to_group = {}
    group_id = 0
    processed = set()

    for name in tqdm(unique_cleaned_names, desc="Fuzzy Matching Progress"):
        if name in processed:
            continue

        # Assign unique group_id to short names (in this way I avoid that all short names are assigned to the same group)
        if len(name) < min_length:
            name_to_group[name] = group_id
            processed.add(name)
            group_id += 1
            continue

        # Use token_sort_ratio for matching longer names and prevent false positives (e.g., "tti" matches "patiperro" in DEU)
        matches = process.extract(
            name,
            unique_cleaned_names,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=similarity_threshold,
            limit=None
        )

        similar_names = [match[0] for match in matches if len(match[0]) >= min_length] # Filter out short names

        if not similar_names:
            name_to_group[name] = group_id
            processed.add(name)
            group_id += 1
            continue

        # Assign group_id to all similar names
        for sim_name in similar_names:
            if sim_name not in name_to_group:
                name_to_group[sim_name] = group_id
                processed.add(sim_name)
        group_id += 1

    # Map group_id back to the DataFrame
    df['group_id'] = df['cleaned_name'].map(name_to_group)
    return df


# Function to assign cleaned_ID based on group_id and country code
def assign_cleaned_id(df, country_code):
    df['cleaned_ID'] = country_code + '_' + df['group_id'].astype(str)
    return df

def process_all_countries(df, similarity_threshold=85):
    final_dfs = []
    countries = df['country_iso3'].unique()
    
    for country in tqdm(countries, desc="Processing Countries"):
        country_df = df[df['country_iso3'] == country].reset_index(drop=True)
        if country_df.empty:
            continue
        # Apply fuzzy matching within the country
        matched_df = fuzzy_match_within_country(country_df, similarity_threshold)
        # Assign cleaned IDs
        assigned_df = assign_cleaned_id(matched_df, country)
        final_dfs.append(assigned_df)
    
    # Combine all country DataFrames
    final_df = pd.concat(final_dfs, ignore_index=True)
    return final_df

In [ ]:
# Apply the process to the entire dataset
final_df = process_all_countries(name_country_data, similarity_threshold=85)

### Machine Learning Model

In [114]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(final_df, test_size=0.3, random_state=42)

In [121]:
## Prepare the Data for Training
# First divide the dataset into two parts: one for training and one for testing (70% training, 30% testing)
train_df, test_df = train_test_split(
    final_df,
    test_size=0.3,
    random_state=42,
    stratify=final_df['country_iso3'] # Stratify by country_iso3 to ensure representation from all countries
)

# Define a corrections dictionary that includes terms from various countries
manual_corrections = {
    'technologies': 'tech',
    'incorporated': 'inc',
    'corporation': 'corp',
    'limited': 'ltd',
    'llc': '',
    'inc': '',
    'gmbh': '',
    'kg': '',
    'sarl': '',
}

def apply_manual_corrections(name):
    words = name.split()
    corrected_words = [manual_corrections.get(word.lower(), word) for word in words]
    # Remove any empty strings resulting from corrections
    corrected_words = [word for word in corrected_words if word]
    return ' '.join(corrected_words)

# Apply manual corrections to the training data
train_df['cleaned_name_manual'] = train_df['cleaned_name'].apply(apply_manual_corrections)

## Machine Learning Model

# Process the data per country to account for country-specific terms

results = []
countries = final_df['country_iso3'].unique() # Get the list of unique countries

for country in countries:
    # Prepare the training data for the current country
    train_country_df = train_df[train_df['country_iso3'] == country]
    test_country_df = test_df[test_df['country_iso3'] == country]
    
    if train_country_df.empty or test_country_df.empty:
        continue  # Skip if there's no data for this country in either set
    
    # Prepare the data for training
    def prepare_training_data(df):
        data = []
        for _, row in df.iterrows():
            foreign_words = re.findall(r'\b\w+\b', row['foreign'].lower()) # Extract words from the foreign name
            cleaned_words = set(row['cleaned_name_manual'].split()) # Extract cleaned words
            for word in foreign_words:
                label = 1 if word in cleaned_words else 0
                data.append({'word': word, 'label': label})
        return pd.DataFrame(data)
    
    training_data = prepare_training_data(train_country_df)
    
    # Simple feature: word length
    training_data['word_length'] = training_data['word'].apply(len)
    
    # Features and target variable
    X_train = training_data[['word_length']]
    y_train = training_data['label']
    
    # Initialize and train the Decision Tree Classifier
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Prepare test data
    def prepare_test_data(df):
        data = []
        index_list = [] 
        for idx, row in df.iterrows():
            foreign_words = re.findall(r'\b\w+\b', row['foreign'].lower())
            for word in foreign_words:
                data.append({'word': word, 'index': idx})
        return pd.DataFrame(data)
    
    test_data = prepare_test_data(test_country_df)
    
    # Extract features for test data
    test_data['word_length'] = test_data['word'].apply(len)
    X_test = test_data[['word_length']]
    
    # Predict whether to keep each word
    test_data['prediction'] = model.predict(X_test)
    
    # Reconstruct the cleaned names based on predictions
    test_country_df = test_country_df.copy()
    test_country_df['cleaned_name_ml'] = ''
    
    for idx in test_country_df.index:
        words = test_data[test_data['index'] == idx]
        kept_words = words[words['prediction'] == 1]['word']
        cleaned_name_ml = ' '.join(kept_words)
        test_country_df.at[idx, 'cleaned_name_ml'] = cleaned_name_ml
    
    # Append the results
    results.append(test_country_df)

# Combine the results from all countries
final_test_df = pd.concat(results, ignore_index=True)



In [ ]:
# Compare the Accuracy Before and After Applying the ML Model

def compare_cleaned_names(row):
    return int(row['cleaned_name_ml'] == apply_manual_corrections(row['cleaned_name']))

final_test_df['correct_cleaning'] = final_test_df.apply(compare_cleaned_names, axis=1)

accuracy = final_test_df['correct_cleaning'].mean()
print(f'Accuracy After Applying Machine Learning: {accuracy:.2f}')

# Print sample results for inspection
print("\nSample Comparison of Cleaned Names:")
print(final_test_df[['foreign', 'cleaned_name', 'cleaned_name_ml']].head())


### Changed Dataset and Export the Results

In [127]:
# Set the columns we are interested in (names_2019_2020 dataset)= ['foreign', 'foreigncountry_cleaned', 'country_iso3', 'cleaned_name', 'cleaned_name_ml']
columns = list(names_2019_2020.columns) + ['cleaned_ID', 'cleaned_name']

# Set the outputfile_alejandro_1 to save the results
outputfile_alejandro_1 = final_df[columns]

# Save the results to a CSV file
outputfile_alejandro_1.to_csv("output/outputfile_alejandro_1.csv", index=False)

In [ ]:
# Evaluate the changes in the naming
# Function to standardize strings for comparison
def standardize_string(s):
    if isinstance(s, str):
        return s.lower().strip()
    else:
        return ''

# Apply the function to 'foreign' and 'cleaned_name' variables
final_df['foreign_standardized'] = final_df['foreign'].apply(standardize_string)
final_df['cleaned_name_standardized'] = final_df['cleaned_name'].apply(standardize_string)

# Identify rows where the names have changed
outputfile_alejandro_1_changed = final_df[final_df['foreign_standardized'] != final_df['cleaned_name_standardized']].copy()

# Include only the original firm name and the cleaned firm name
outputfile_alejandro_1_changed = outputfile_alejandro_1_changed[['foreign', 'cleaned_name']]

# Save the changed names to a CSV file
outputfile_alejandro_1_changed.to_csv("output/outputfile_alejandro_1_changed.csv", index=False)


## Part 2

In [ ]:
# Import data
names_2021 = pd.read_csv("data/ForeignNames_2021.csv")

# Fix Congo in the names_2021 dataset and apply manual mapping
names_2021['foreigncountry_cleaned'] = names_2021['foreigncountry_cleaned'].replace('Congo, the Democratic Republic of the','Congo')

# Apply the manual mapping to the 'foreigncountry_cleaned' column
names_2021['name_clean'] = names_2021['foreigncountry_cleaned'].replace(manual_country_mapping)

# Now we can merge based on the mapped country names
names_2021 = pd.merge(names_2021, country_iso, 
                      left_on='foreigncountry_cleaned', 
                      right_on='name_clean', 
                      how='left', indicator=True)

names_2021._merge.value_counts()


In [129]:
# Drop the rows where the company name is missing
names_2021 = names_2021.dropna(subset=['foreign'])
# Apply the cleaning function
names_2021['cleaned_name'] = names_2021.apply(
    lambda row: clean_company_name(
        row['foreign'],
        countries_stopwords.get(row['country_iso3'], [])
    ),
    axis=1
)

In [131]:
# Prepare the old data, data from 2019-2020
old_data = final_df[['cleaned_name', 'cleaned_ID', 'country_iso3', 'foreign']].copy()
old_data = old_data.drop_duplicates(subset=['cleaned_name', 'country_iso3'])

# Prepare the new data, data from 2021
new_data = names_2021[['foreign', 'cleaned_name', 'country_iso3']].copy()

# Set 'cleaned_ID', 'new', and 'old_name'
new_data['cleaned_ID'] = np.nan
new_data['new'] = 1  # Assume new firms initially
new_data['old_name'] = np.nan

# Function to match and assign IDs
def match_and_assign_ids(row):
    country = row['country_iso3']
    cleaned_name = row['cleaned_name']
    
    # Filter old data for the same country
    old_data_country = old_data[old_data['country_iso3'] == country]
    
    if old_data_country.empty:
        return row  # No old data for this country
    
    # Use RapidFuzz to find the best match
    match = process.extractOne(
        cleaned_name,
        old_data_country['cleaned_name'],
        scorer=fuzz.token_sort_ratio, # Use token_sort_ratio for better matching
        score_cutoff=85  # Similarity threshold
    )
    
    if match:
        # Match found
        matched_cleaned_name = match[0]
        matched_row = old_data_country[old_data_country['cleaned_name'] == matched_cleaned_name].iloc[0] # Get the first match
        row['cleaned_ID'] = matched_row['cleaned_ID']
        row['new'] = 0
        row['old_name'] = matched_row['foreign']
    else:
        # No match found, assign new ID
        max_id = old_data_country['cleaned_ID'].str.extract(rf'{country}_(\d+)')[0].astype(float).max() # Extract the ID number and get the max
        if pd.isna(max_id):
            max_id = 0
        new_id_number = int(max_id) + 1
        row['cleaned_ID'] = f"{country}_{new_id_number}"
    return row

# Apply the function row-wise
new_data = new_data.apply(match_and_assign_ids, axis=1)

# Select and reorder the required columns
outputfile_alejandro_2 = new_data[['foreign', 'cleaned_name', 'cleaned_ID', 'new', 'old_name']]

# Export the results to a CSV file
outputfile_alejandro_2.to_csv("output/outputfile_alejandro_2.csv", index=False)